In [2]:
# !pip install -U langchain
!pip install langchain langchain-community langchain-core
!pip install langchain-openai openai
!pip install langchain-deepseek
!pip install deepseek-api python-dotenv

# Installing the OpenAI integration
#!pip install -U langchain-openai
# Installing the Anthropic integration
# !pip install -U langchain-anthropic

!pip show langchain-core
!pip install langchain-mcp-adapters

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/is

In [3]:
import os
from langchain.tools import tool, ToolRuntime
from langchain_openai import ChatOpenAI


# 设置 DeepSeek 的 API 密钥（LangChain-OpenAI 仍然会查找 OPENAI_API_KEY）
os.environ['DEEPSEEK_API_KEY'] = 'sk'

# 您可以尝试将 DEEPSEEK_API_KEY 的值赋给 OPENAI_API_KEY 环境变量
os.environ["OPENAI_API_KEY"] = os.getenv('DEEPSEEK_API_KEY') # 确保这个值是 DeepSeek 的 key

# 关键：指定 DeepSeek 的 API 基础 URL
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"

model = ChatOpenAI(
    model="deepseek-chat", # 使用 DeepSeek 的模型名称
    openai_api_base=DEEPSEEK_BASE_URL, # 指定 DeepSeek 的 URL
    temperature=0.7
)

# os.environ["OPENAI_API_KEY"]

In [5]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware 
from langgraph.checkpoint.memory import InMemorySaver 


agent = create_agent(
    model=model,
    # tools=[write_file_tool, execute_sql_tool, read_data_tool],
    middleware=[
        HumanInTheLoopMiddleware( 
            interrupt_on={
                "write_file": True,  # All decisions (approve, edit, reject) allowed
                "execute_sql": {"allowed_decisions": ["approve", "reject"]},  # No editing allowed
                # Safe operation, no approval needed
                "read_data": False,
            },
            # Prefix for interrupt messages - combined with tool name and args to form the full message
            # e.g., "Tool execution pending approval: execute_sql with query='DELETE FROM...'"
            # Individual tools can override this by specifying a "description" in their interrupt config
            description_prefix="Tool execution pending approval",
        ),
    ],
    # Human-in-the-loop requires checkpointing to handle interrupts.
    # In production, use a persistent checkpointer like AsyncPostgresSaver.
    checkpointer=InMemorySaver(),  
)

In [10]:
from langgraph.types import Command

# Human-in-the-loop leverages LangGraph's persistence layer.
# You must provide a thread ID to associate the execution with a conversation thread,
# so the conversation can be paused and resumed (as is needed for human review).
config = {"configurable": {"thread_id": "some_id"}} 
# Run the graph until the interrupt is hit.
result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Delete old records from the database",
            }
        ]
    },
    config=config 
)

# The interrupt contains the full HITL request with action_requests and review_configs
# print(result['__interrupt__'])

# Resume with approval decision
agent.invoke(
    Command( 
        resume={"decisions": [{"type": "approve"}]}  # or "edit", "reject"
    ), 
    config=config # Same thread ID to resume the paused conversation
)

{'messages': [HumanMessage(content='Delete old records from the database', additional_kwargs={}, response_metadata={}, id='57e16dd0-0556-4584-b46f-9208024702ed'),
  HumanMessage(content='Delete old records from the database', additional_kwargs={}, response_metadata={}, id='b5fe92ff-a0f2-4487-b9b8-28c82c9c805d'),
  AIMessage(content='I\'ll help you with deleting old records from a database. Here are several approaches depending on your database system and requirements:\n\n## SQL Examples\n\n### 1. MySQL/MariaDB\n```sql\n-- Delete records older than 30 days\nDELETE FROM your_table \nWHERE created_at < DATE_SUB(NOW(), INTERVAL 30 DAY);\n\n-- Delete with limit to avoid locking issues\nDELETE FROM your_table \nWHERE created_at < \'2023-01-01\' \nLIMIT 1000;\n```\n\n### 2. PostgreSQL\n```sql\n-- Delete records older than 30 days\nDELETE FROM your_table \nWHERE created_at < NOW() - INTERVAL \'30 days\';\n\n-- Using date column\nDELETE FROM your_table \nWHERE date_column < CURRENT_DATE - 30;\n

In [ ]:
agent.invoke(
    Command(
        # Decisions are provided as a list, one per action under review.
        # The order of decisions must match the order of actions
        # listed in the `__interrupt__` request.
        resume={
            "decisions": [
                {
                    "type": "approve",
                }
            ]
        }
    ),
    config=config  # Same thread ID to resume the paused conversation
)